# Writing ansible plugins

## Ansible filters

Ansible already comes with a large collection of [filters](https://docs.ansible.com/ansible/latest/playbook_guide/playbooks_filters.html).
For example

```yml
{{ some_variable | to_nice_json }}
```

can be used to automatically create and deploy config files based on gathered variables.

However sometimes it is either very cumbersome or not possible to achieve certain tasks and a small custom filter plugin might be very useful.

### Example: `to_dot_env` to automatically format .env files

We want to be able to do the following:

In [ ]:
from IPython.display import Code
Code(filename="playbook.yml", language="yaml")

As a result the `.env` file should look like

```
PYTHONPATH=/var/service/lib/python/site_packages
PATH=/var/service/bin
```

### Where to start?
From the [ansible doc](https://docs.ansible.com/ansible/latest/plugins/filter.html):
> You can add a custom filter plugin by dropping it into a `filter_plugins` directory adjacent to your play, inside
> a role, or by putting it in one of the filter plugin directory sources configured in [ansible.cfg](https://docs.ansible.com/ansible/latest/reference_appendices/config.html#ansible-configuration-settings).


### Step 1: Create `filter_plugins` folder

Given the playbook `playbook.yml`, create a folder `filter_plugins` next to the playbook.
And create a file `to_dot_env.py` inside.

```
playbook.yml
filter_plugins/
└── to_dot_env.py
```

In [ ]:
!mkdir filter_plugins

### Step 2: Write a filter class

Open `to_dot_env.py`,

Put the following inside:

In [ ]:
%%writefile filter_plugins/to_dot_env.py


class FilterModule:
    @staticmethod
    def filters():
        return {"to_dot_env": to_dot_env}


def to_dot_env(keys):
    # your code here
    return "..."

Lets check what the filter currently does:

In [ ]:
%%script bash --no-raise-error

ansible-playbook playbook.yml

It should have written something to `/tmp/.env`. Lets check the content:

In [ ]:
!cat /tmp/.env

If ansible finds a python file in the folder `filter_plugins`, it first looks if the file contains a class `FilterModule` with a method `filters`. It uses this method as an index of filter plugins inside the python file.
In our case it is `to_dot_env` and it calls the function with the same name in the python file.

If in the playbook `... | to_dot_env` is used, then ansible will call `to_dot_env(...)` passing to the function whatever `...` evaluates to.

Tasks:
- [ ] Make sure that in `to_dot_env(keys)`, `keys` is a `dict`. Otherwise rise a `TypeError` exception.
- [ ] After this check, convert the `dict` to a `str` of the form
  ```
  key1=val1
  key2=val2
  ...
  ```
  and return this string
- [ ] Run the playbook to check, that the filter works

# Ansible modules

Ansible modules are what you use in your playbooks:
```yml
- name: Use module
  ansible.builtin.copy: # <- copy is a module
  ...
```

### Example 1: Hello world

Source: https://auscunningham.medium.com/write-a-ansible-module-with-python-527f0b292b4d

First, we create a playbook which will call our plugin `hello_world`:

In [ ]:
from IPython.display import Code
Code(filename="helloworld.yml", language="yaml")

The following module just will return a field `msg` in its return value.

In [ ]:
!mkdir -p library

In [ ]:
%%writefile library/hello_world.py
#!/bin/env python3

from ansible.module_utils.basic import AnsibleModule

def main():
    module = AnsibleModule(argument_spec={})
    module.exit_json(changed=False, msg="Chuck Norris killed Bambi")

if __name__ == "__main__":
    main()

Call `ansible-playbook` to test the module:

In [ ]:
%%script bash --no-raise-error

ansible-playbook helloworld.yml

### Example 2: Assemble a config file starting from an example config

Note: this also can by some ansible modules and some filters, but here we develop one simple (for the enduser) module.

Assume a os package (e.g. installed py `ansible.builtin.apt`) comes with some example config in `/var/lib/pkg/example.json`.

Goal: use this as a template, fill in some values passed by a parameter.

Our template config will be:

In [ ]:
from IPython.display import Code
Code(filename="example.json", language="json")

And our playbook will pass some values to fill in:

In [ ]:
from IPython.display import Code
Code(filename="assemble_config.yml", language="yaml")

In [ ]:
!mkdir -p library

In [ ]:
%%writefile library/assemble_config.py
#!/bin/env python3

from ansible.module_utils.basic import AnsibleModule

DOCUMENTATION = """---
module: assemble_config.py
short_description: ...
description:
  - ...
options:
  template:
    description:
      - ...
  ...
"""

def main():
    module = AnsibleModule(argument_spec={
        "template": {"required": True, "type": "str"},
        "dest": {"required": True, "type": "str"},
        "values": {"required": True, "type": "dict"},
    })
    changed = assemble_config(**module.params)
    module.exit_json(changed=changed)

def assemble_config(template: str, dest: str, values: dict) -> bool:
    # Your code here
    return False
    
if __name__ == "__main__":
    main()

In [ ]:
%%script bash --no-raise-error

ansible-playbook assemble_config.yml